In [1]:
import asyncio
from typing import List, Optional
from datetime import datetime

from fastapi import Depends
from pydantic import BaseModel, Field

from vexa import VexaAPI
from meeting_search import VectorSearch
from core import system_msg, user_msg, assistant_msg, generic_call_stream, count_tokens, BaseCall
from prompts import Prompts
from pydantic_models import ThreadName, ContextQualityCheck
from assistant_thread import ThreadManager

In [8]:
class SearchAssistant:
    def __init__(self, user_token: Optional[str] = None):
        self.vexa = VexaAPI(user_token) if user_token else VexaAPI()
        self.analyzer = VectorSearch()
        self.thread_manager = ThreadManager()
        self.prompts = Prompts()
        
    def get_thread(self, thread_id: str):
        return self.thread_manager.get_thread(thread_id)

    def get_user_threads(self):
        return self.thread_manager.get_user_threads(self.vexa.user_id)

    def count_documents(self):
        return self.analyzer.count_documents(user_id=self.vexa.user_id)

    def get_messages_by_thread_id(self, thread_id: str):
        return self.thread_manager.get_messages_by_thread_id(thread_id)

In [12]:
thread_id = None


In [13]:
self = SearchAssistant()

User information retrieved successfully.


In [45]:
from typing import Union, Any, Dict
from pydantic_models import QueryPlan
from meeting_search import build_context_string


In [32]:
query = 'list customer names'


In [33]:
user_id = self.vexa.user_id
user_name = self.vexa.user_name

if thread_id:
    thread = self.thread_manager.get_thread(thread_id)
    if not thread:
        raise ValueError(f"Thread with id {thread_id} not found")
    messages = thread.messages
    thread_name = thread.thread_name
else:
    messages = []
    thread_name = None

In [34]:
query_ = ' '.join([m.content for m in messages]) + ' ' + query
queries = await self.analyzer.generate_search_queries(query_, user_id=user_id, user_name=user_name)

In [37]:
self = self.analyzer

In [46]:
async def generate_search_queries(self, query: str,last_n_meetings:int=100,user_id: Optional[Union[str, List[str]]] = None, user_name: Optional[Union[str, List[str]]] = None) -> List[Dict[str, Any]]:
    summaries = self.get_summaries(user_id=user_id, user_name=user_name)[-last_n_meetings:]
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    planner_prompt = """think step by step of which of the following meetings are relevant to the user request. and create many (5+) requests to the search system which will be used to find similar meetings to the user request, 
                        requests should have at least 125 char each and at least 2 sentences."""
    
    plan = await QueryPlan.call([
        system_msg(f"""{planner_prompt}.
                        Now is {now}.
                        Write queries based on user request and general context you have, don't use start and end if timing is not obvious from the user query.
                    """),
        user_msg(f'general context: last {last_n_meetings} meeting summaries in ascending order: {build_context_string(summaries,only_summaries=True)}'),
        user_msg(f'user request: {query}.')
    ])

    # Extract queries from the plan
    queries = plan[0].model_dump()['queries']
    return queries

In [48]:
last_n_meetings = 100

In [49]:
summaries = self.get_summaries(user_id=user_id, user_name=user_name)[-last_n_meetings:]

In [53]:
build_context_string(summaries,only_summaries=True)[0]

"# [1] AI Applications in Business Processes (2024-09-12 20:46):\nIn a meeting on September 12, 2024, Sergey Ryabenko and Dmitriy Grankin discussed the use of AI in generating acceptance criteria and summaries. Sergey highlighted Toptal's use of AI for client documentation, while Dmitriy presented his product that improves summaries iteratively. Both emphasized the practical benefits of AI in enhancing business processes and client interactions.\n\n---\n\n# [2] Team Discussion on Resources (2024-09-12 21:26):\nDmitriy Grankin leads a casual team discussion addressing resource shortages and potential solutions. The conversation includes a mix of languages, reflecting team diversity, and highlights the need for efficiency and collaboration.\n\n---\n\n# [3] Landing Page Optimization Discussion (2024-09-12 22:34):\nOn September 12, 2024, Olga Nemirovskaya and Dmitriy Grankin discussed the optimization of landing pages for their product, focusing on user feedback, visual updates, and market

In [43]:
queries

[{'start': None, 'end': None, 'vector_search_query': 'customer names'},
 {'start': None, 'end': None, 'vector_search_query': 'client names'},
 {'start': None, 'end': None, 'vector_search_query': 'list of customers'},
 {'start': None, 'end': None, 'vector_search_query': 'customer information'},
 {'start': None, 'end': None, 'vector_search_query': 'client details'}]